In [23]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import time

from bs4 import BeautifulSoup
import requests

In [24]:
url = 'https://gradesfixer.com'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html')

In [25]:
df_category = pd.DataFrame()

categorical_urls =  [value['href'] for value in soup.find('li').find_all('a')]
categorical_urls = categorical_urls[:-1]

df_category['url'] = categorical_urls
df_category

,url
0,https://gradesfixer.com/free-essay-examples/art/
1,https://gradesfixer.com/free-essay-examples/bu...
2,https://gradesfixer.com/free-essay-examples/ec...
3,https://gradesfixer.com/free-essay-examples/ed...
4,https://gradesfixer.com/free-essay-examples/en...
5,https://gradesfixer.com/free-essay-examples/en...
6,https://gradesfixer.com/free-essay-examples/go...
7,https://gradesfixer.com/free-essay-examples/he...
8,https://gradesfixer.com/free-essay-examples/hi...
9,https://gradesfixer.com/free-essay-examples/in...


In [26]:
df_category.reset_index(inplace=True)
df_category

,index,url
0,0,https://gradesfixer.com/free-essay-examples/art/
1,1,https://gradesfixer.com/free-essay-examples/bu...
2,2,https://gradesfixer.com/free-essay-examples/ec...
3,3,https://gradesfixer.com/free-essay-examples/ed...
4,4,https://gradesfixer.com/free-essay-examples/en...
5,5,https://gradesfixer.com/free-essay-examples/en...
6,6,https://gradesfixer.com/free-essay-examples/go...
7,7,https://gradesfixer.com/free-essay-examples/he...
8,8,https://gradesfixer.com/free-essay-examples/hi...
9,9,https://gradesfixer.com/free-essay-examples/in...


In [27]:
pages_numbers_l = []

for url_cat in df_category['url']:
    page = requests.get(url_cat)
    soup = BeautifulSoup(page.text, 'html')

    pages_numbers_l.append(int(soup.find_all(class_='page-numbers')[3].text))

df_category['pages'] = pages_numbers_l
df_category

IndexError: list index out of range

In [ ]:
pages_dict = {'FK_category':[], 'url': []}

for index in df_category.index:
    url = df_category.loc[index, 'url']
    pages = df_category.loc[index, 'pages']

    pages_dict['FK_category'] += [index]*pages
    pages_dict['url'] += [f"{url}page/{i}" for i in range(pages)]

pages_df = pd.DataFrame(pages_dict)
pages_df

In [ ]:
essays_df_dict = {'FK_page': [], "url": []}

n_samples = 600

for index in pages_df.sample(n_samples).index:
    url_page = pages_df.loc[index, 'url']
    page = requests.get(url_page)
    soup = BeautifulSoup(page.text, 'html')

    articles = soup.find_all('article', class_='loop-10 related-essay-item')

    for article in articles:
        essays_df_dict['FK_page'] += [index]
        essays_df_dict['url'] += [article.find('h2').find('a')['href']]

essays_df = pd.DataFrame(essays_df_dict)
essays_df

In [ ]:
texts_l = []

for url in essays_df['url']:
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html')

    by_div_class = soup.find('div', class_='single-essay-item')
    if by_div_class is None:
        texts_l.append(None)
        continue
    by_div_class_p_l = by_div_class.find_all(['p'])#, 'h2'

    result_text = ''
    for text_peace in by_div_class_p_l:
        if not text_peace.parent.has_key('class'): continue
        elif text_peace.parent['class'][0] == 'single-essay-item':
            text = text_peace.text

            result_text += text
            result_text += '\n'

    texts_l.append(result_text)

    #time_to_wait = np.abs(np.random.normal(loc=0.1, scale=0.01))
    #time.sleep(time_to_wait)

essays_df['text'] = texts_l
essays_df

In [ ]:
len(texts_l)

In [ ]:
essays_df['text'].to_csv('human_essays.csv', index=False)

In [ ]:
len(essays_df['text'])

In [ ]:
print(essays_df['text'].sample().iloc[0])